https://haystack.deepset.ai/tutorials/01_basic_qa_pipeline

In [1]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,inference]

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
from haystack.telemetry import tutorial_running
tutorial_running(1)

/home/addo/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [19]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)

INFO - haystack.modeling.utils -  Using devices: CPU - Number of GPUs: 0


In [12]:
doc_dir = "data/wiki_gameofthrones_txt1"

In [20]:
import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline

files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
indexing_pipeline = TextIndexingPipeline(document_store)
indexing_pipeline.run_batch(file_paths=files_to_index)

INFO - haystack.pipelines.base -  It seems that an indexing Pipeline is run, so using the nodes' run method instead of run_batch.
Preprocessing:  39%|███▉      | 72/183 [00:00<00:00, 180.99docs/s]WARNING - haystack.nodes.preprocessor.preprocessor -  Document dd048b8e5bcb7de1be5bd3937f15442f is 14232 characters long after preprocessing, where the maximum length should be 10000. Something might be wrong with the splitting, check the document affected to prevent issues at query time. This document will be now hard-split at 10000 chars recursively.
WARNING - haystack.nodes.preprocessor.preprocessor -  Document b48fb0da693eb4d81b3566d0069868b3 is 10488 characters long after preprocessing, where the maximum length should be 10000. Something might be wrong with the splitting, check the document affected to prevent issues at query time. This document will be now hard-split at 10000 chars recursively.
Updating BM25 representation...: 100%|██████████| 2359/2359 [00:00<00:00, 17223.88 docs/s]


{'documents': [<Document: {'content': '\n\nThe \'\'\'Dothraki language\'\'\' is a constructed fictional language in George R. R. Martin\'s fantasy novel series \'\'A Song of Ice and Fire\'\' and its television adaptation \'\'Game of Thrones\'\'. It is spoken by the Dothraki, a nomadic people in the series\'s fictional world. The language was developed for the TV series by the linguist David J. Peterson, working off the Dothraki words and phrases in Martin\'s novels.\n\n, the language comprised 3163 words, not all of which have been made public. In 2012, 146 newborn girls in the United States were named "Khaleesi", the Dothraki term for the wife of a \'\'khal\'\' or ruler, and the title adopted in the series by Daenerys Targaryen. Dothraki and Valyrian have been described as "the most convincing fictional tongues since Elvish".\n\n==Development==\nDavid J. Peterson, creator of the Dothraki spoken language for \'\'Game of Thrones\'\'\n\nThe Dothraki vocabulary was created by David J. Pet

In [21]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

https://huggingface.co/deepset/roberta-base-squad2

In [22]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO - haystack.modeling.utils -  Using devices: CPU - Number of GPUs: 0
INFO - haystack.modeling.utils -  Using devices: CPU - Number of GPUs: 0
INFO - haystack.modeling.model.language_model -   * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)
INFO - haystack.modeling.model.language_model -  Auto-detected model language: english
INFO - haystack.modeling.model.language_model -  Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.
INFO - haystack.modeling.utils -  Using devices: CPU - Number of GPUs: 0


In [23]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [24]:
prediction = pipe.run(
    query="Who is the father of Arya Stark?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples: 100%|██████████| 1/1 [00:06<00:00,  6.15s/ Batches]


In [29]:
from pprint import pprint

# pprint(prediction)

In [28]:
from haystack.utils import print_answers

print_answers(prediction, details="medium")  ## Choose from `minimum`, `medium`, and `all`

'Query: Who is the father of Arya Stark?'
'Answers:'
[   {   'answer': 'Eddard',
        'context': 's Nymeria after a legendary warrior queen. She travels '
                   "with her father, Eddard, to King's Landing when he is made "
                   'Hand of the King. Before she leaves,',
        'score': 0.993372917175293},
    {   'answer': 'Ned',
        'context': 'k in the television series.\n'
                   '\n'
                   '====Season 1====\n'
                   'Arya accompanies her father Ned and her sister Sansa to '
                   "King's Landing. Before their departure, Arya's h",
        'score': 0.9753611087799072},
    {   'answer': 'Lord Eddard Stark',
        'context': 'rk daughters.\n'
                   '\n'
                   'During the Tourney of the Hand to honour her father Lord '
                   'Eddard Stark, Sansa Stark is enchanted by the knights '
                   'performing in the event.',
        'score': 0.9177320599555969}